In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
import os

sns.set_style("darkgrid")

In [2]:
df = pd.read_excel(r"D:\Files\DataSets\2019Cricket\2019ODICricketMatches.xlsx") 

In [53]:
df.describe() 

<ipython-input-53-dd1f68e3e9d1>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe()


,Team 1,Team 2,Winner,Looser,Lost By,Margin,Win by Runs,Win by Wickets,Ground,Match Date,Scorecard
count,128,128,128,123,121,123,63,58,128,128,128
unique,17,16,20,18,2,63,53,8,50,102,128
top,Afghanistan,Pakistan,England,Zimbabwe,Runs,5 wickets,3 runs,5 wickets,Harare,2018-03-06 00:00:00,ODI # 3972
freq,16,18,17,20,63,10,4,10,16,4,1
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-01-06 00:00:00,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-14 00:00:00,NaN


In [4]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Team 1      128 non-null    object        
 1   Team 2      128 non-null    object        
 2   Winner      128 non-null    object        
 3   Margin      123 non-null    object        
 4   Ground      128 non-null    object        
 5   Match Date  128 non-null    datetime64[ns]
 6   Scorecard   128 non-null    object        
dtypes: datetime64[ns](1), object(6)
memory usage: 7.1+ KB


# Transform margin column into two columns. One for wickets and the other for wins. 

In [5]:
df.head() 
    

,Team 1,Team 2,Winner,Margin,Ground,Match Date,Scorecard
0,New Zealand,Pakistan,New Zealand,61 runs,Wellington,2018-01-06,ODI # 3946
1,New Zealand,Pakistan,New Zealand,8 wickets,Nelson,2018-01-09,ODI # 3947
2,U.A.E.,Ireland,Ireland,4 wickets,ICCA Dubai,2018-01-11,ODI # 3948
3,New Zealand,Pakistan,New Zealand,183 runs,Dunedin,2018-01-13,ODI # 3949
4,U.A.E.,Ireland,Ireland,67 runs,ICCA Dubai,2018-01-13,ODI # 3950


In [6]:
def runs(margin):
    try:
        if "runs" in margin:
            return margin
        else: return np.nan
    except TypeError:
        return np.nan


In [7]:
df["Margin"].isna().sum() 

5

In [8]:
winruns = df["Margin"].apply(runs) 

In [9]:
def wickets(margin):
    try:
        if "wickets" in margin:
            return margin
        else: return np.nan
    except TypeError:
        return np.nan

In [10]:
winwickets = df["Margin"].apply(wickets) 

In [11]:
df.insert(df.columns.get_loc("Margin") + 1, "Win by Runs", winruns)
df.insert(df.columns.get_loc("Margin") + 2, "Win by Wickets", winwickets) 

In [12]:
df 

,Team 1,Team 2,Winner,Margin,Win by Runs,Win by Wickets,Ground,Match Date,Scorecard
0,New Zealand,Pakistan,New Zealand,61 runs,61 runs,NaN,Wellington,2018-01-06,ODI # 3946
1,New Zealand,Pakistan,New Zealand,8 wickets,NaN,8 wickets,Nelson,2018-01-09,ODI # 3947
2,U.A.E.,Ireland,Ireland,4 wickets,NaN,4 wickets,ICCA Dubai,2018-01-11,ODI # 3948
3,New Zealand,Pakistan,New Zealand,183 runs,183 runs,NaN,Dunedin,2018-01-13,ODI # 3949
4,U.A.E.,Ireland,Ireland,67 runs,67 runs,NaN,ICCA Dubai,2018-01-13,ODI # 3950
...,...,...,...,...,...,...,...,...,...
123,Australia,South Africa,South Africa,40 runs,40 runs,NaN,Hobart,2018-11-11,ODI # 4069
124,New Zealand,Pakistan,no result,NaN,NaN,NaN,Dubai (DSC),2018-11-11,ODI # 4070
125,Bangladesh,West Indies,Bangladesh,5 wickets,NaN,5 wickets,Dhaka,2018-12-09,ODI # 4071
126,Bangladesh,West Indies,West Indies,4 wickets,NaN,4 wickets,Dhaka,2018-12-11,ODI # 4072


# Which country played the most ODI in 2018? 

In [13]:
pd.Series(df["Team 1"].tolist() + df["Team 2"].tolist()).value_counts().idxmax()

'Zimbabwe'

# Top 3 countries who won the most ODIs

In [14]:
df.groupby(["Winner"])["Winner"].value_counts().sort_values(ascending=False).head(3) 

Winner      Winner    
England     England       17
India       India         14
Bangladesh  Bangladesh    13
Name: Winner, dtype: int64

# How was the performance of Sri Lanka?

In [15]:
filt = df["Winner"] == "Sri Lanka"

In [16]:
# General wins
df.loc[filt]["Margin"].count() 

6

In [17]:
# Wins by runs 
df.loc[filt]["Win by Runs"].count()

4

In [18]:
# Wins by wickets
df.loc[filt]["Win by Wickets"].count() 

2

In [19]:
# Monthly general wins
df.set_index("Match Date").groupby("Winner").resample("m")["Winner"].count()["Sri Lanka"] 

Match Date
2018-01-31    3
2018-02-28    0
2018-03-31    0
2018-04-30    0
2018-05-31    0
2018-06-30    0
2018-07-31    0
2018-08-31    2
2018-09-30    0
2018-10-31    1
Name: Winner, dtype: int64

In [20]:
# Monthly wins by wickets
df.set_index("Match Date").groupby("Winner").resample("m")["Win by Wickets"].count()["Sri Lanka"] 

Match Date
2018-01-31    2
2018-02-28    0
2018-03-31    0
2018-04-30    0
2018-05-31    0
2018-06-30    0
2018-07-31    0
2018-08-31    0
2018-09-30    0
2018-10-31    0
Name: Win by Wickets, dtype: int64

In [21]:
# Monthly wins by wickets
df.set_index("Match Date").groupby("Winner").resample("m")["Win by Runs"].count()["Sri Lanka"] 

Match Date
2018-01-31    1
2018-02-28    0
2018-03-31    0
2018-04-30    0
2018-05-31    0
2018-06-30    0
2018-07-31    0
2018-08-31    2
2018-09-30    0
2018-10-31    1
Name: Win by Runs, dtype: int64

In [22]:
# Win rate
count = 0
for country in (df["Team 1"].tolist() + df["Team 2"].tolist()):
    if country == "Sri Lanka":
        count += 1
print(df.loc[filt]["Margin"].count() / count) 

0.35294117647058826


# Top 3 wins by runs 

In [23]:
df.groupby(["Winner"])["Win by Runs"].count().sort_values(ascending = False).head(3) 

Winner
Bangladesh     8
England        8
Afghanistan    5
Name: Win by Runs, dtype: int64

# Month in which most ODIs were played

In [24]:
df.set_index("Match Date").resample("m")["Scorecard"].count().sort_values(ascending=False).idxmax() 

Timestamp('2018-03-31 00:00:00')

# How many cricket matches were played every month? 

In [25]:
df.set_index("Match Date").resample("m")["Scorecard"].count()

Match Date
2018-01-31    23
2018-02-28    13
2018-03-31    26
2018-04-30     0
2018-05-31     0
2018-06-30     6
2018-07-31    12
2018-08-31    10
2018-09-30    14
2018-10-31    14
2018-11-30     7
2018-12-31     3
Freq: M, Name: Scorecard, dtype: int64

# On which ground most games were played? 

In [26]:
df.columns 

Index(['Team 1', 'Team 2', 'Winner', 'Margin', 'Win by Runs', 'Win by Wickets',
       'Ground', 'Match Date', 'Scorecard'],
      dtype='object')

In [27]:
df.groupby(["Ground"])["Scorecard"].count().idxmax()  

'Harare'

# Did India win mostly by wins or wickets? 

In [28]:
filt = df["Winner"] == "India"

In [29]:
India_df = df.loc[filt] 

In [30]:
India_df

,Team 1,Team 2,Winner,Margin,Win by Runs,Win by Wickets,Ground,Match Date,Scorecard
23,South Africa,India,India,6 wickets,NaN,6 wickets,Durban,2018-02-01,ODI # 3969
24,South Africa,India,India,9 wickets,NaN,9 wickets,Centurion,2018-02-04,ODI # 3970
25,South Africa,India,India,124 runs,124 runs,NaN,Cape Town,2018-02-07,ODI # 3971
30,South Africa,India,India,73 runs,73 runs,NaN,Port Elizabeth,2018-02-13,ODI # 3976
32,South Africa,India,India,8 wickets,NaN,8 wickets,Centurion,2018-02-16,ODI # 3978
68,England,India,India,8 wickets,NaN,8 wickets,Nottingham,2018-07-12,ODI # 4014
93,Hong Kong,India,India,26 runs,26 runs,NaN,Dubai (DSC),2018-09-18,ODI # 4039
94,India,Pakistan,India,8 wickets,NaN,8 wickets,Dubai (DSC),2018-09-19,ODI # 4040
96,Bangladesh,India,India,7 wickets,NaN,7 wickets,Dubai (DSC),2018-09-21,ODI # 4042
98,India,Pakistan,India,9 wickets,NaN,9 wickets,Dubai (DSC),2018-09-23,ODI # 4044


In [31]:
if India_df["Win by Runs"].count()  > India_df["Win by Wickets"].count():
    print("India won mostly by Runs")
elif India_df["Win by Runs"].count()  == India_df["Win by Wickets"].count():
    print("India won by an equal number of wins and runs")
else:
    print("India won mostly by wickets") 

India won mostly by wickets


# Top 3 countries which won most games and what was their win percentage every month

In [32]:
df["Winner"].value_counts().sort_values(ascending = False).head(3) 

England       17
India         14
Bangladesh    13
Name: Winner, dtype: int64

In [33]:
England_wins = df.set_index("Match Date").groupby("Winner").resample("m")["Winner"].count()["England"] 
India_wins = df.set_index("Match Date").groupby("Winner").resample("m")["Winner"].count()["India"] 
Bangladesh_wins = df.set_index("Match Date").groupby("Winner").resample("m")["Winner"].count()["Bangladesh"]  

In [34]:
T1E = df.set_index("Match Date").groupby(["Team 1"]).resample("m")["Team 1"].count()["England"] 
T2E = df.set_index("Match Date").groupby(["Team 2"]).resample("m")["Team 2"].count()["England"] 
England_matches = T2E.add(T1E, fill_value=0) 

In [35]:
T1I = df.set_index("Match Date").groupby(["Team 1"]).resample("m")["Team 1"].count()["India"] 
T2I = df.set_index("Match Date").groupby(["Team 2"]).resample("m")["Team 2"].count()["India"] 
India_matches = T2I.add(T1I, fill_value=0) 

In [36]:
T1B = df.set_index("Match Date").groupby(["Team 1"]).resample("m")["Team 1"].count()["Bangladesh"] 
T2B = df.set_index("Match Date").groupby(["Team 2"]).resample("m")["Team 2"].count()["Bangladesh"] 
Bangladesh_matches = T2B.add(T1B, fill_value=0) 

In [37]:
England_winrate = England_wins.div(England_matches, fill_value=0)
India_winrate = India_wins.div(India_matches, fill_value=0)
Bangladesh_winrate = Bangladesh_wins.div(Bangladesh_matches, fill_value=0) 

In [38]:
England_winrate

Match Date
2018-01-31    0.800000
2018-02-28    0.500000
2018-03-31    0.666667
2018-04-30         NaN
2018-05-31         NaN
2018-06-30    0.833333
2018-07-31    0.666667
2018-08-31         NaN
2018-09-30         NaN
2018-10-31    0.600000
dtype: float64

In [39]:
India_winrate

Match Date
2018-02-28    0.833333
2018-03-31         NaN
2018-04-30         NaN
2018-05-31         NaN
2018-06-30         NaN
2018-07-31    0.333333
2018-08-31         NaN
2018-09-30    0.833333
2018-10-31    0.500000
2018-11-30    1.000000
dtype: float64

In [40]:
Bangladesh_winrate 

Match Date
2018-01-31    0.600000
2018-02-28         NaN
2018-03-31         NaN
2018-04-30         NaN
2018-05-31         NaN
2018-06-30         NaN
2018-07-31    0.666667
2018-08-31         NaN
2018-09-30    0.500000
2018-10-31    1.000000
2018-11-30         NaN
2018-12-31    0.666667
dtype: float64

# Team which had most lost matches

In [41]:
teams = (df["Team 1"] + "," + df["Team 2"] + "," + df["Winner"])

In [42]:
def team(teams):
    teams = teams.split(",")
    if teams[0] == teams[2]:
        return teams[1]
    elif teams[1] == teams[2]:
        return teams[0]
    else:
        return np.nan

In [43]:
looser = teams.apply(team) 

In [44]:
df.insert(df.columns.get_loc("Winner") + 1, "Looser", looser) 

In [45]:
df.groupby("Looser")["Looser"].count().idxmax() 

'Zimbabwe'

## Did this team lose most matches by runs or wickets? 

In [46]:
df 

,Team 1,Team 2,Winner,Looser,Margin,Win by Runs,Win by Wickets,Ground,Match Date,Scorecard
0,New Zealand,Pakistan,New Zealand,Pakistan,61 runs,61 runs,NaN,Wellington,2018-01-06,ODI # 3946
1,New Zealand,Pakistan,New Zealand,Pakistan,8 wickets,NaN,8 wickets,Nelson,2018-01-09,ODI # 3947
2,U.A.E.,Ireland,Ireland,U.A.E.,4 wickets,NaN,4 wickets,ICCA Dubai,2018-01-11,ODI # 3948
3,New Zealand,Pakistan,New Zealand,Pakistan,183 runs,183 runs,NaN,Dunedin,2018-01-13,ODI # 3949
4,U.A.E.,Ireland,Ireland,U.A.E.,67 runs,67 runs,NaN,ICCA Dubai,2018-01-13,ODI # 3950
...,...,...,...,...,...,...,...,...,...,...
123,Australia,South Africa,South Africa,Australia,40 runs,40 runs,NaN,Hobart,2018-11-11,ODI # 4069
124,New Zealand,Pakistan,no result,NaN,NaN,NaN,NaN,Dubai (DSC),2018-11-11,ODI # 4070
125,Bangladesh,West Indies,Bangladesh,West Indies,5 wickets,NaN,5 wickets,Dhaka,2018-12-09,ODI # 4071
126,Bangladesh,West Indies,West Indies,Bangladesh,4 wickets,NaN,4 wickets,Dhaka,2018-12-11,ODI # 4072


In [47]:
def loses(lose):
    try:
        if "runs" in lose:
            return "Runs"
        elif "wickets" in lose:
            return "Wickets"
    except TypeError:
        return np.nan

In [48]:
lostby = df["Margin"].apply(loses) 

In [49]:
df.insert(df.columns.get_loc("Looser") + 1, "Lost By", lostby)

In [50]:
filt = (df["Looser"] == "Zimbabwe")

In [51]:
df.loc[filt]["Lost By"].value_counts().idxmax() 

'Wickets'